## Ref 

- https://cloud.google.com/vision/
- https://cloud.google.com/vision/docs/how-to
- https://pypi.org/project/google-cloud-vision/

In [1]:
# python 3 

# gcloud 
from google.cloud import vision
from google.oauth2 import service_account
from google.protobuf.json_format import MessageToDict

# OP 
import pandas as pd 
import numpy as np 

#--------------------------------------------------
# config 
# save ur google cloud credentials below
credentials = service_account.Credentials.from_service_account_file('/Users/yennanliu/Downloads/google_cloud_creds2.json')
client = vision.ImageAnnotatorClient(credentials=credentials)


In [2]:

#--------------------------------------------------
# help func 

def call_google_image_api(url):
    response = client.annotate_image({
    'image': {'source': {'image_uri': url}},
    'features': [{'type': vision.enums.Feature.Type.FACE_DETECTION}],})
    print ('response : ', response)
    response_dict = MessageToDict(response)
    return response_dict




def call_google_handwritten_api(uri):

    """
    # ref 
    https://cloud.google.com/vision/docs/ocr

    # put ur image url above 
    # e.g. https://s-i.huffpost.com/gen/2165258/images/n-NY-TIMES-ERROR-628x314.jpg

    """

    """Detects handwritten characters in the file located in Google Cloud
    Storage.

    Args:
    uri: The path to the file in Google Cloud Storage (gs://...)
    """
    from google.cloud import vision_v1p3beta1 as vision
    credentials = service_account.Credentials.from_service_account_file('google_cloud_creds2.json')
    
    
    client = vision.ImageAnnotatorClient(credentials=credentials)
    image = vision.types.Image()
    image.source.image_uri = uri

    # Language hint codes for handwritten OCR:
    # en-t-i0-handwrit, mul-Latn-t-i0-handwrit
    # Note: Use only one language hint code per request for handwritten OCR.
    image_context = vision.types.ImageContext(
        language_hints=['en-t-i0-handwrit'])

    response = client.document_text_detection(image=image,
                                              image_context=image_context)

    print('Full Text: {}'.format(response.full_text_annotation.text))
    for page in response.full_text_annotation.pages:
        for block in page.blocks:
            print('\nBlock confidence: {}\n'.format(block.confidence))

            for paragraph in block.paragraphs:
                print('Paragraph confidence: {}'.format(
                    paragraph.confidence))

                for word in paragraph.words:
                    word_text = ''.join([
                        symbol.text for symbol in word.symbols
                    ])
                    print('Word text: {} (confidence: {})'.format(
                        word_text, word.confidence))

                    for symbol in word.symbols:
                        print('\tSymbol: {} (confidence: {})'.format(
                            symbol.text, symbol.confidence))


#--------------------------------------------------


In [3]:
####### run on one uri #######

uri = 'https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg'

print ('uri :' , uri)
response = call_google_image_api(uri)

uri : https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg
response :  face_annotations {
  bounding_poly {
    vertices {
      x: 277
      y: 34
    }
    vertices {
      x: 425
      y: 34
    }
    vertices {
      x: 425
      y: 206
    }
    vertices {
      x: 277
      y: 206
    }
  }
  fd_bounding_poly {
    vertices {
      x: 288
      y: 73
    }
    vertices {
      x: 387
      y: 73
    }
    vertices {
      x: 387
      y: 172
    }
    vertices {
      x: 288
      y: 172
    }
  }
  landmarks {
    type: LEFT_EYE
    position {
      x: 312.9682922363281
      y: 105.73651123046875
      z: 0.00100697239395231
    }
  }
  landmarks {
    type: RIGHT_EYE
    position {
      x: 347.94708251953125
      y: 104.90869903564453
      z: -23.364290237426758
    }
  }
  landmarks {
    type: LEFT_OF_LEFT_EYEBROW
    position {
      x: 301.98956298828125
      y: 96.25361633300781
     

In [4]:
type(response)

dict

In [7]:
# transform google cloud vision api response to dict 
# https://stackoverflow.com/questions/19734617/protobuf-to-json-in-python

#from google.protobuf.json_format import MessageToDict
#dict_obj = MessageToDict(response)


In [8]:
####### run on uri dataframe  #######

url_list = ['https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg',
            'https://ichef.bbci.co.uk/news/660/cpsprodpb/1344B/production/_98032987_gettyimages-810310530.jpg',
            'https://www.factsninja.com/wp-content/uploads/2017/12/Steve-Jobs-action-figure-portrait.jpg',
            'https://www.ikea.com/gb/en/images/products/fj%C3%A4llbo-laptop-table-black__0476347_pe616247_s4.jpg']



In [10]:
url_list

['https://cbsnews1.cbsistatic.com/hub/i/2018/09/24/bbee124f-bbe8-4d73-80c3-7e681214e062/0924-cbsn-trumpkimsummit-ayfayg-1665758-640x360.jpg',
 'https://ichef.bbci.co.uk/news/660/cpsprodpb/1344B/production/_98032987_gettyimages-810310530.jpg',
 'https://www.factsninja.com/wp-content/uploads/2017/12/Steve-Jobs-action-figure-portrait.jpg',
 'https://www.ikea.com/gb/en/images/products/fj%C3%A4llbo-laptop-table-black__0476347_pe616247_s4.jpg']

In [11]:
df_query_result = pd.DataFrame({ 'url':url_list })

In [12]:
df_query_result

,url
0,https://cbsnews1.cbsistatic.com/hub/i/2018/09/...
1,https://ichef.bbci.co.uk/news/660/cpsprodpb/13...
2,https://www.factsninja.com/wp-content/uploads/...
3,https://www.ikea.com/gb/en/images/products/fj%...


In [13]:
df_query_result['query_dict'] = df_query_result['url'].apply(lambda x : call_google_image_api(x))

#call_google_image_api()

response :  face_annotations {
  bounding_poly {
    vertices {
      x: 277
      y: 34
    }
    vertices {
      x: 425
      y: 34
    }
    vertices {
      x: 425
      y: 206
    }
    vertices {
      x: 277
      y: 206
    }
  }
  fd_bounding_poly {
    vertices {
      x: 288
      y: 73
    }
    vertices {
      x: 387
      y: 73
    }
    vertices {
      x: 387
      y: 172
    }
    vertices {
      x: 288
      y: 172
    }
  }
  landmarks {
    type: LEFT_EYE
    position {
      x: 312.9682922363281
      y: 105.73651123046875
      z: 0.00100697239395231
    }
  }
  landmarks {
    type: RIGHT_EYE
    position {
      x: 347.94708251953125
      y: 104.90869903564453
      z: -23.364290237426758
    }
  }
  landmarks {
    type: LEFT_OF_LEFT_EYEBROW
    position {
      x: 301.98956298828125
      y: 96.25361633300781
      z: 13.67840576171875
    }
  }
  landmarks {
    type: RIGHT_OF_LEFT_EYEBROW
    position {
      x: 317.3613586425781
      y: 93.505706787109

response :  face_annotations {
  bounding_poly {
    vertices {
      x: 214
    }
    vertices {
      x: 444
    }
    vertices {
      x: 444
      y: 266
    }
    vertices {
      x: 214
      y: 266
    }
  }
  fd_bounding_poly {
    vertices {
      x: 246
      y: 88
    }
    vertices {
      x: 408
      y: 88
    }
    vertices {
      x: 408
      y: 250
    }
    vertices {
      x: 246
      y: 250
    }
  }
  landmarks {
    type: LEFT_EYE
    position {
      x: 301.0611572265625
      y: 136.93162536621094
      z: 0.0015892947558313608
    }
  }
  landmarks {
    type: RIGHT_EYE
    position {
      x: 364.46087646484375
      y: 137.5692901611328
      z: 2.6101138591766357
    }
  }
  landmarks {
    type: LEFT_OF_LEFT_EYEBROW
    position {
      x: 273.9918212890625
      y: 121.81608581542969
      z: 1.069809079170227
    }
  }
  landmarks {
    type: RIGHT_OF_LEFT_EYEBROW
    position {
      x: 315.8761901855469
      y: 126.57849884033203
      z: -16.7388725

In [14]:
df_query_result

,url,query_dict
0,https://cbsnews1.cbsistatic.com/hub/i/2018/09/...,{'faceAnnotations': [{'sorrowLikelihood': 'VER...
1,https://ichef.bbci.co.uk/news/660/cpsprodpb/13...,{'faceAnnotations': [{'sorrowLikelihood': 'VER...
2,https://www.factsninja.com/wp-content/uploads/...,{'faceAnnotations': [{'sorrowLikelihood': 'VER...
3,https://www.ikea.com/gb/en/images/products/fj%...,{}


In [15]:
df_query_result.iloc[2]['query_dict']

{'faceAnnotations': [{'angerLikelihood': 'VERY_UNLIKELY',
   'blurredLikelihood': 'VERY_UNLIKELY',
   'boundingPoly': {'vertices': [{'x': 214},
     {'x': 444},
     {'x': 444, 'y': 266},
     {'x': 214, 'y': 266}]},
   'detectionConfidence': 0.9968630075454712,
   'fdBoundingPoly': {'vertices': [{'x': 246, 'y': 88},
     {'x': 408, 'y': 88},
     {'x': 408, 'y': 250},
     {'x': 246, 'y': 250}]},
   'headwearLikelihood': 'VERY_UNLIKELY',
   'joyLikelihood': 'VERY_UNLIKELY',
   'landmarkingConfidence': 0.4248863756656647,
   'landmarks': [{'position': {'x': 301.0611572265625,
      'y': 136.93162536621094,
      'z': 0.0015892947558313608},
     'type': 'LEFT_EYE'},
    {'position': {'x': 364.46087646484375,
      'y': 137.5692901611328,
      'z': 2.6101138591766357},
     'type': 'RIGHT_EYE'},
    {'position': {'x': 273.9918212890625,
      'y': 121.81608581542969,
      'z': 1.069809079170227},
     'type': 'LEFT_OF_LEFT_EYEBROW'},
    {'position': {'x': 315.8761901855469,
      'y'